In [1]:
import feedparser
import time
import requests
import re

from tqdm import tqdm
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
feed_urls = [
    "https://www.dealnews.com/c142/Electronics/?rss=1",
        # "https://www.dealnews.com/c39/Computers/?rss=1",
        # "https://www.dealnews.com/c238/Automotive/?rss=1",
        # "https://www.dealnews.com/f1912/Smart-Home/?rss=1",
        # "https://www.dealnews.com/c196/Home-Garden/?rss=1",
       ]

In [3]:
def extract(html_snippet):
    soup = BeautifulSoup(html_snippet, 'html.parser')
    snippet_div = soup.find('div', class_='snippet summary')
    
    if snippet_div:
        description = snippet_div.get_text(strip=True)
        description = BeautifulSoup(description, 'html.parser').get_text()
        description = re.sub('<[^<]+?>', '', description)
        result = description.strip()
    else:
        result = html_snippet
    return result.replace('\n', ' ')

In [4]:
def parse_entry(entry):
    parsed = {}
    parsed['title'] = entry['title']
    parsed['summary'] = extract(entry['summary'])
    parsed['url'] = entry['links'][0]['href']
    stuff = requests.get(parsed['url']).content
    soup = BeautifulSoup(stuff, 'html.parser')
    content = soup.find('div', class_='content-section').get_text()
    content = content.replace('\nmore', '').replace('\n', ' ').strip()
    if "Features" in content:
        details, features = content.split("Features")
    else:
        details = content
        features = ""
    # parsed['content'] = content
    parsed['details'] = details
    parsed['features'] = features.strip()
    
    return parsed

In [7]:
def fetch(show_progress=False):
    deals = []
    feed_iter = tqdm(feed_urls) if show_progress else feed_urls
    for feed_url in feed_iter:
        feed = feedparser.parse(feed_url)
        pprint(feed.entries[0])
        for entry in feed.entries[:10]:
            deals.append(parse_entry(entry))
            time.sleep(0.5)
    return deals

In [8]:
deals = fetch(False)

{'guidislink': False,
 'id': 'https://www.dealnews.com/21808584.html?iref=rss-c142',
 'link': 'https://www.dealnews.com/Hisense-32-A45-NFC-32-1080-p-FHD-LED-Smart-TV-for-79-free-shipping/21808584.html?iref=rss-c142',
 'links': [{'href': 'https://www.dealnews.com/Hisense-32-A45-NFC-32-1080-p-FHD-LED-Smart-TV-for-79-free-shipping/21808584.html?iref=rss-c142',
            'rel': 'alternate',
            'type': 'text/html'}],
 'published': 'Sun, 08 Feb 2026 09:49:43 -0500',
 'published_parsed': time.struct_time(tm_year=2026, tm_mon=2, tm_mday=8, tm_hour=14, tm_min=49, tm_sec=43, tm_wday=6, tm_yday=39, tm_isdst=0),
 'summary': '<img '
            'src="https://c.dlnws.com/image/upload/f_auto,t_large,q_auto/cms/a6j5tlgcgwcolo2oewne.jpg" '
            'style="float: left; vertical-align: top; margin: 0 8px 8px 0;" '
            '/><div class="snippet summary" '
            'title="Use&#x20;promo&#x20;code&#x20;&quot;EBAYVIP2026SALE&quot;&#x20;to&#x20;drop&#x20;it&#x20;to&#x20;&#x24;79&#x3B;&

In [8]:
print(len(deals))

10


In [9]:
print(type(deals[0]))

<class 'dict'>


In [10]:
for deal in deals:
    pprint(deal)
    print()

{'details': 'Use promo code "EBAYVIP2026SALE" to drop it to $79; a total '
            'savings of $21 off. Buy Now at eBay',
 'features': '',
 'summary': 'Use promo code "EBAYVIP2026SALE" to drop it to $79; a total '
            'savings of $21 off. Buy Now at eBay',
 'title': 'Hisense 32A45NFC 32" 1080p FHD LED Smart TV for $79 + free shipping',
 'url': 'https://www.dealnews.com/Hisense-32-A45-NFC-32-1080-p-FHD-LED-Smart-TV-for-79-free-shipping/21808584.html?iref=rss-c142'}

{'details': 'Use promo code "EBAYVIP2026SALE" to drop the price. That\'s a '
            '$100 savings. Buy Now at eBay',
 'features': '',
 'summary': 'Use promo code "EBAYVIP2026SALE" to drop the price. That\'s a '
            '$100 savings. Buy Now at eBay',
 'title': 'Unlocked Motorola G 5G 128GB Smartphone for $68 + free shipping',
 'url': 'https://www.dealnews.com/products/Motorola/Unlocked-Motorola-G-5-G-128-GB-Smartphone/497576.html?iref=rss-c142'}

{'details': 'Use promo code "EBAYVIP2026SALE" to drop the